In [ ]:
#ee version 0.1.221
##geemap version 0.6.13

In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

In [2]:
print(ee.__version__)
print(geemap.__version__)

0.1.221
0.6.13


In [5]:
manaus_shp = '/Users/morgansteckler/Desktop/SULI 2020/tile_shapefiles/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
#manaus_shp = '/home/6ru/tile_shapefiles/manaus/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

In [ ]:
atto_shp = '/Users/morgansteckler/Desktop/Work/tile_shapefiles/venus_atto_footprint/venus_atto_footprint.shp'
#atto_shp = '/home/6ru/tile_shapefiles/atto/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)
info = atto.geometry()
atto = ee.Geometry(info)

In [ ]:
corumba_shp = '/Users/morgansteckler/Desktop/Work/tile_shapefiles/venus_corumba_footprint/venus_corumba_footprint.shp'
#corumba_shp = '/home/6ru/tile_shapefiles/corumba/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)
info = corumba.geometry()
corumba = ee.Geometry(info)

In [ ]:
mato_shp = '/Users/morgansteckler/Desktop/Work/tile_shapefiles/venus_mato_footprint/venus_mato_footprint.shp'
#mato_shp = '/home/6ru/tile_shapefiles/mato/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)
info = mato.geometry()
mato = ee.Geometry(info)

In [ ]:
saop_shp = '/Users/morgansteckler/Desktop/Work/tile_shapefiles/venus_saop_footprint/venus_saop_footprint.shp'
#saop_shp = '/home/6ru/tile_shapefiles/saop/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)
info = saop.geometry()
saop = ee.Geometry(info)

In [6]:
#Select geom of interest to export
fc = manaus

In [7]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

In [8]:
#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int() #bit 10 is clouds
cirrusBitMask = ee.Number(2).pow(11).int() #bit 11 is cirrus clouds
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [9]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2')#Select S2 1-C level data
      .select(['B5', 'B8', 'QA60']) #Select necessary bands to reduce memory usage
      .filterBounds(fc) #Narrow data spatially
      .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

In [10]:
#Create collection with cloud mask
col = s2.map(maskS2clouds)

In [11]:
#15-day step
years = ee.List.sequence(2016, 2019)
step = ee.List.sequence(1, 365, 15)

def byYear(y):
    y = ee.Number(y)
    def byStep(d):
        d = ee.Number(d)
        return (col
                .filter(ee.Filter.calendarRange(y, y.add(1), 'year')) #yearly step
                .filter(ee.Filter.calendarRange(d, d.add(15), 'day_of_year')) #15-day step
                .qualityMosaic('NDRE') #Max greenness
                .set('step', [d, y])) #Add properties
    return step.map(byStep)

col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
col = col.map(lambda image: image.clip(fc)).select('NDRE')

In [ ]:
##ONLY RUN THIS BLOCK FOR ATTO, OTHERWISE SKIP##
#Atto removal: 5, 24
col = (col.filter(ee.Filter.neq('step', [76, 2016])) #5
       .filter(ee.Filter.neq('step', [361, 2016]))) #24

In [ ]:
##ONLY RUN THIS BLOCK FOR SAOP, OTHERWISE SKIP##
#Saop removal: 9, 24, 99
col = (col.filter(ee.Filter.neq('step', [361, 2019])) #99
       .filter(ee.Filter.neq('step', [361, 2016])) #24
       .filter(ee.Filter.neq('step', [136, 2016]))) #9

In [12]:
#Export images from a collection
out_dir = '/Users/morgansteckler/Desktop/'
geemap.ee_export_image_collection(col, out_dir, region = fc, scale = 20, file_per_band=False)

Total number of images: 100

Exporting 1/100: 0.tif
Generating URL ...
Please wait ...


KeyboardInterrupt: 